In [2]:
import pandas as pd
from sklearn import datasets
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import mlflow
from mlflow.models import infer_signature

In [3]:
mlflow.set_tracking_uri("http://localhost:5000")

In [7]:
##loading dataset
X,y=datasets.load_iris(return_X_y=True)
# split data into sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

#define model hyperparameters- considering these are the best(hyperameter tuning)
params={"penalty":'l2', "solver":'lbfgs', "max_iter":1000, "random_state":8888}

#train model
print("Training the model...")
lr=LogisticRegression(**params)
lr.fit(X_train,y_train)


Training the model...


,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,8888
,solver,'lbfgs'
,max_iter,1000
,multi_class,'deprecated'


In [8]:
y_pred=lr.predict(X_test)
y_pred

array([1, 0, 0, 0, 0, 0, 2, 1, 0, 0, 1, 2, 2, 2, 0, 2, 1, 1, 0, 0, 2, 2,
       2, 1, 0, 0, 1, 1, 2, 0])

In [12]:
print(y_test)
print(y_pred)
accuracy=accuracy_score(y_test,y_pred)
accuracy

[1 0 0 0 0 0 2 1 0 0 1 2 2 2 0 2 1 1 0 0 2 2 2 1 0 0 1 1 2 0]
[1 0 0 0 0 0 2 1 0 0 1 2 2 2 0 2 1 1 0 0 2 2 2 1 0 0 1 1 2 0]


1.0

In [50]:
### MLflow tracking
mlflow.set_tracking_uri("http://localhost:5000")

## new experiment
mlflow.set_experiment("MLFLOW Quickstart")

## start mlflow run
with mlflow.start_run():
    mlflow.log_params(params) #logging hyperameters
    mlflow.log_metric("accuracy", accuracy)
    signature=infer_signature(X_train, lr.predict(X_train)) #infer model signature/schema
    model_info=mlflow.sklearn.log_model(
        sk_model=lr,
        name="iris model",
        signature=signature,
        input_example=X_train,
        registered_model_name="tracking quickstart"
    ) # can find the parameters in documentation

Registered model 'tracking quickstart' already exists. Creating a new version of this model...
2025/11/04 17:28:05 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: tracking quickstart, version 5


🏃 View run likeable-duck-649 at: http://localhost:5000/#/experiments/811392785418817872/runs/2e87ab6535414d62b0fa33c1f195cada
🧪 View experiment at: http://localhost:5000/#/experiments/811392785418817872


Created version '5' of model 'tracking quickstart'.


In [42]:
new_X_train, new_X_test, new_y_train, new_y_test = train_test_split(X, y, test_size=0.2, random_state=48)
new_y_pred=lr.predict(X_test)
new_y_pred

array([1, 0, 0, 0, 0, 0, 2, 1, 0, 0, 1, 2, 2, 2, 0, 2, 1, 1, 0, 0, 2, 2,
       2, 1, 0, 0, 1, 1, 2, 0])

In [43]:
lr.fit(new_X_train,new_y_train)
y_pred=lr.predict(X_test)
y_pred
new_accuracy=accuracy_score(new_y_pred,new_y_test)
new_accuracy

0.3

In [51]:
with mlflow.start_run():
    mlflow.log_params(params) #logging hyperameters
    mlflow.log_metric("accuracy", new_accuracy)
    signature=infer_signature(new_X_train, lr.predict(new_X_train)) #infer model signature/schema
    model_info=mlflow.sklearn.log_model(
        sk_model=lr,
        name="iris model",
        signature=signature,
        input_example=new_X_train,
        registered_model_name="tracking quickstart2"
    ) # can find the parameters in documentation

Successfully registered model 'tracking quickstart2'.
2025/11/04 17:28:22 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: tracking quickstart2, version 1


🏃 View run orderly-cub-714 at: http://localhost:5000/#/experiments/811392785418817872/runs/cee41474f74c47ceb49a2315706b351d
🧪 View experiment at: http://localhost:5000/#/experiments/811392785418817872


Created version '1' of model 'tracking quickstart2'.


In [52]:
model_info.model_uri

'models:/m-929af95c0ba94642952c426556d85812'

### Inferencing and validating model

In [53]:
from mlflow.models import validate_serving_input
model_uri=model_info.model_uri
#serving_payload="""{"inputs":[[5.3,4.4],[1.4,5.6]]}""" #use proper input if want to perform
#validate_serving_input(model_uri,serving_payload)

### Load model back for prediction as a generic python function model

In [54]:
loaded_model=mlflow.pyfunc.load_model(model_info.model_uri)
predictions=loaded_model.predict(X_test)

iris_feature_name=datasets.load_iris().feature_names
result=pd.DataFrame(X_test, columns=iris_feature_name)
result["actual_class"]=y_test
result["predicted_class"]=predictions

In [55]:
result

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),actual_class,predicted_class
0,6.1,2.8,4.0,1.3,1,1
1,4.8,3.4,1.9,0.2,0,0
2,5.0,3.5,1.6,0.6,0,0
3,5.3,3.7,1.5,0.2,0,0
4,4.7,3.2,1.3,0.2,0,0
5,5.2,4.1,1.5,0.1,0,0
6,6.2,3.4,5.4,2.3,2,2
7,6.2,2.2,4.5,1.5,1,1
8,4.9,3.0,1.4,0.2,0,0
9,5.0,3.6,1.4,0.2,0,0


### Model Registry

centralized model store, set of API's, UI to manage lifecycle of a model. Provides model lineage, versioning, aliasing, tagging etc.

can comare your models and see whats best and register that, add tags and aliases

In [60]:
model_name="tracking quickstart"
model_version="latest"
model_uri = f"models:/{model_name}/{model_version}"

model=mlflow.sklearn.load_model(model_uri)
model

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,8888
,solver,'lbfgs'
,max_iter,1000
,multi_class,'deprecated'


In [63]:
y_pred_new=model.predict(X_test)
y_pred_new

array([1, 0, 0, 0, 0, 0, 2, 1, 0, 0, 1, 2, 2, 2, 0, 2, 1, 1, 0, 0, 2, 2,
       2, 1, 0, 0, 1, 1, 2, 0])